In [12]:
import os
import requests
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
from psycopg2.extras import execute_values
from utils import get_db_connection, log

# Load environment
load_dotenv()
NASA_URL = os.getenv("NASA_FIRMS_URL")
API_KEY = os.getenv("NASA_FIRMS_API_KEY")

# Expected columns matching the DB schema
EXPECTED_COLUMNS = [
    "latitude", "longitude", "brightness", "scan", "track",
    "acq_date", "acq_time", "satellite", "confidence", "version",
    "bright_t31", "frp", "daynight"
]

def fetch_firms_df(country="USA", days=3, product="MODIS_NRT"):
    url = f"{NASA_URL}/{API_KEY}/{product}/{country}/{days}"
    log(f"Fetching FIRMS data: {url}")
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"HTTP error {response.status_code}: {response.text}")
    return pd.read_csv(StringIO(response.text))

def load_df_to_postgres(df):
    if df.empty:
        log("⚠️ DataFrame is empty. No data to insert.")
        return

    # Standardize column names
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # Retain only expected columns
    df = df[[col for col in EXPECTED_COLUMNS if col in df.columns]]

    with get_db_connection() as conn:
        with conn.cursor() as cur:
            log("Creating table satellite_signals if not exists...")
            cur.execute("""
                CREATE TABLE IF NOT EXISTS satellite_signals (
                    latitude FLOAT,
                    longitude FLOAT,
                    brightness FLOAT,
                    scan FLOAT,
                    track FLOAT,
                    acq_date DATE,
                    acq_time TEXT,
                    satellite TEXT,
                    confidence TEXT,
                    version TEXT,
                    bright_t31 FLOAT,
                    frp FLOAT,
                    daynight TEXT
                );
            """)
            conn.commit()

            # Prepare and insert rows
            values = [tuple(row) for row in df.to_numpy()]
            insert_sql = f"""
                INSERT INTO satellite_signals ({', '.join(EXPECTED_COLUMNS)})
                VALUES %s
            """
            log(f"Inserting {len(values)} rows using execute_values...")
            execute_values(cur, insert_sql, values)
        conn.commit()
    log("✔ Fast insert completed successfully.")

if __name__ == "__main__":
    df = fetch_firms_df()
    load_df_to_postgres(df)


[LOG] Fetching FIRMS data: https://firms.modaps.eosdis.nasa.gov/api/country/csv/0525bda225833e502171d32a59c8464d/MODIS_NRT/USA/3
[LOG] Creating table satellite_signals if not exists...
[LOG] Inserting 2066 rows using execute_values...
[LOG] ✔ Fast insert completed successfully.


In [13]:
df

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,USA,65.21597,-153.41406,356.67,1.65,1.26,2025-07-13,33,Aqua,MODIS,98,6.1NRT,301.56,152.34,D
1,USA,65.21635,-153.42178,376.02,1.65,1.26,2025-07-13,33,Aqua,MODIS,100,6.1NRT,301.69,273.94,D
2,USA,65.21885,-153.37589,360.09,1.65,1.26,2025-07-13,33,Aqua,MODIS,87,6.1NRT,293.44,172.00,D
3,USA,65.21902,-153.38448,382.23,1.65,1.26,2025-07-13,33,Aqua,MODIS,94,6.1NRT,295.46,327.65,D
4,USA,65.22500,-153.45953,325.27,1.65,1.26,2025-07-13,33,Aqua,MODIS,50,6.1NRT,300.83,34.97,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,USA,30.94340,-88.00817,302.84,1.20,1.10,2025-07-15,846,Aqua,MODIS,95,6.1URT,290.36,5.61,N
2062,USA,42.69297,-111.58763,318.49,1.50,1.20,2025-07-15,1021,Aqua,MODIS,75,6.1URT,285.03,34.38,N
2063,USA,45.47971,-114.98827,306.02,2.50,1.50,2025-07-15,1021,Aqua,MODIS,95,6.1URT,288.20,27.84,N
2064,USA,45.48375,-114.99261,309.50,2.50,1.50,2025-07-15,1021,Aqua,MODIS,95,6.1URT,288.44,35.41,N
